In [ ]:
%matplotlib widget
import os
from plantcv import plantcv as pcv
from plantcv.parallel import WorkflowInputs

In [ ]:
args = WorkflowInputs(images=["./imgs/10.9.1.29_pos-153-001-003_2019-10-26-19-05.jpg"],
                      names="image1",
                      result="data_output.json",
                      outdir="img_output",
                      writeimg=False,
                      debug="plot")

In [ ]:
pcv.params.debug_outdir = args.outdir
pcv.params.debug = args.debug
pcv.params.text_size=50
pcv.params.text_thickness=15

In [ ]:
img, path, filename = pcv.readimage(filename=args.image1)


In [ ]:
#### Create a mask and begin to clean it ####
# Change color space for plant extraction
gray_img = pcv.rgb2gray_lab(rgb_img=img, channel="a")


In [ ]:
# Threshold
bin_img = pcv.threshold.binary(gray_img=gray_img, threshold=120, object_type='dark')


In [ ]:
# Fill in small objects 
flt_mask = pcv.fill(bin_img=bin_img, size=50)


In [ ]:
# Dilate 
clean_mask = pcv.dilate(gray_img=flt_mask, ksize=2, i=1)

In [ ]:
tray_roi = pcv.roi.rectangle(img=img, x=950, y=0, h=2464, w=2100)

In [ ]:
tray_mask = pcv.roi.filter(mask=clean_mask, roi=tray_roi)

In [ ]:
# Create a grid of ROIs 
rois = pcv.roi.auto_grid(mask=tray_mask, nrows=6, ncols=4)

In [ ]:
labeled_mask, n_labels = pcv.create_labels(mask=tray_mask, rois=rois)

In [ ]:
shape_img = pcv.analyze.size(img=img, labeled_mask=labeled_mask, n_labels=n_labels, label="plant")

In [ ]:
if args.writeimg:
    pcv.print_image(shape_img, os.path.join(args.outdir, f"{filename}_shapes.jpg"))


In [ ]:
pcv.outputs.save_results(filename=args.result)